# Data Cleaning and Import into mySQL Database

#### DATA 311 | Patrick Harrison

In [1]:
import pandas as pd
import numpy as np
from pprint import pprint

from mysql import connector
from mysql.connector import Error

import logging
logging.getLogger(__name__)
logging.basicConfig(filename="DataImport.log",
                    level=logging.INFO,
                    format="[%(asctime)s] %(levelname)s: %(message)s",
                    filemode="w")
logging.info("Import File Started")

<hr>

### Import the data into pandas dataframe object

In [2]:
data = pd.read_csv("KeplarData_NcolDesc.csv", header=0)
data.head()

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_vet_stat,koi_vet_date,koi_pdisposition,koi_score,koi_fpflag_nt,...,koi_dicco_mdec,koi_dicco_mdec_err,koi_dicco_msky,koi_dicco_msky_err,koi_dikco_mra,koi_dikco_mra_err,koi_dikco_mdec,koi_dikco_mdec_err,koi_dikco_msky,koi_dikco_msky_err
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,Done,2018-08-16,CANDIDATE,1.000,0,...,0.200,0.160,0.200,0.170,0.080,0.130,0.310,0.170,0.320,0.160
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,Done,2018-08-16,CANDIDATE,0.969,0,...,0.000,0.480,0.390,0.360,0.490,0.340,0.120,0.730,0.500,0.450
2,3,10811496,K00753.01,NaN,CANDIDATE,Done,2018-08-16,CANDIDATE,0.000,0,...,-0.034,0.070,0.042,0.072,0.002,0.071,-0.027,0.074,0.027,0.074
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,Done,2018-08-16,FALSE POSITIVE,0.000,0,...,0.147,0.078,0.289,0.079,-0.257,0.072,0.099,0.077,0.276,0.076
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,Done,2018-08-16,CANDIDATE,1.000,0,...,-0.090,0.180,0.100,0.140,0.070,0.180,0.020,0.160,0.070,0.200


The complete data is now stored in a pandas dataframe object. We will create two new dataframes for each of the future MySQL tables with the wanted columns for each respective dataframe.

In [3]:
# List of attributes wanted for each table
koiObjectsCols = ["kepoi_name", "kepid", "koi_disposition", "koi_score"]
transitPropertiesCols = ["kepoi_name", "kepid", "koi_period", "koi_prad",\
                         "koi_sma", "koi_teq", "koi_ror", "koi_dor"]
stellarObjectsCols = ["kepid", "koi_steff", "koi_slogg", "koi_smet",\
                      "koi_srad", "koi_smass", "koi_srho", "koi_count"]

#### Kepler Objects of Interest (koi) table

The koi table will contain all the koi objects observed in this dataset. It will store the unique_id (`kepid`), the unique kepler name (`kepoi_name`) which contains the host star and the unique transit object number, whether the transit object is confirmed to exist (`koi_disposition`) and the confidence level that objects are exoplanets (`koi_score`).

The `kepoi_name` is of the form
$$\text{KXXXXX.DD}$$
where XXXXX denoted the host star id, and DD denotes the unique transit object.

In [4]:
koiObjects = data[koiObjectsCols].copy()

In [5]:
koiObjects.head()

,kepoi_name,kepid,koi_disposition,koi_score
0,K00752.01,10797460,CONFIRMED,1.000
1,K00752.02,10797460,CONFIRMED,0.969
2,K00753.01,10811496,CANDIDATE,0.000
3,K00754.01,10848459,FALSE POSITIVE,0.000
4,K00755.01,10854555,CONFIRMED,1.000


#### Transit Properties Dataframe Creation

The transit properties dataframe contains information for the transit objects and thier orbits and physical properties. Each transit object is given a unique kepler identification number `kepid`. Just like in the koi_objects table, we will use this as the primary key.

In [6]:
# Create the Transit object dataframe
transitProperties = data[transitPropertiesCols].copy()

In [7]:
transitProperties.head()

,kepoi_name,kepid,koi_period,koi_prad,koi_sma,koi_teq,koi_ror,koi_dor
0,K00752.01,10797460,9.488036,2.26,0.0853,793.0,0.022344,24.810
1,K00752.02,10797460,54.418383,2.83,0.2734,443.0,0.027954,77.900
2,K00753.01,10811496,19.899140,14.60,0.1419,638.0,0.154046,53.500
3,K00754.01,10848459,1.736952,33.46,0.0267,1395.0,0.387394,3.278
4,K00755.01,10854555,2.525592,2.75,0.0374,1406.0,0.024064,8.750


#### Stellar Object Dataframe Creation

To create the `stellar_objects` dataframe, use the same process as for the `transit_object` dataframe, but then remove duplicate rows on the `kepid` attribute.

In [8]:
stellarObjects = data[stellarObjectsCols].copy()

In [9]:
# Drop the duplicates
stellarObjects.drop_duplicates(subset="kepid", inplace=True)
stellarObjects.head()

,kepid,koi_steff,koi_slogg,koi_smet,koi_srad,koi_smass,koi_srho,koi_count
0,10797460,5455.0,4.467,0.14,0.927,0.919,3.20796,2
2,10811496,5853.0,4.544,-0.18,0.868,0.961,7.29555,1
3,10848459,5805.0,4.564,-0.52,0.791,0.836,0.22080,1
4,10854555,6031.0,4.438,0.07,1.046,1.095,1.98635,1
5,10872983,6046.0,4.486,-0.08,0.972,1.053,0.67324,3


## Cleaning

#### Transit Properties Dataframe

The transit object dataframe should not contain any duplicate primary keys. In the context of this dataset, a duplicate primary key would mean a duplicate observation. This is not helpful and would mess up the counts.

In [10]:
# Make sure there are no duplicate rows based on kepid (unique transit object)
transitProperties.drop_duplicates(subset="kepoi_name", inplace=True)

The Rows with all null values or NaN values are not going to be helpful. However, If some attributes are filled, kepping the row is still acceptable. The data is transit properties. If some properties are unknown, the rest of the data is likely to still be accurate.

In [11]:
# Drop rows with all null values
transitProperties.dropna(how='all', inplace=True)
transitProperties.replace(np.NaN, None, inplace=True) # Change np.NaN -> None
#transitProperties[transitProperties.isna().any(axis=1)]

kepoi_name    None
kepid         None
koi_period    None
koi_prad      None
koi_sma       None
koi_teq       None
koi_ror       None
koi_dor       None
dtype: object

In [12]:
# Type assertion
print(transitProperties.dtypes)

kepoi_name     object
kepid           int64
koi_period    float64
koi_prad      float64
koi_sma       float64
koi_teq       float64
koi_ror       float64
koi_dor       float64
dtype: object


### Stellar Objects Dataframe

All duplicate rows have already been removed when the table was created.

In [13]:
# Drop rows with null values
stellarObjects.dropna(how='all', inplace=True)
stellarObjects.replace(np.NaN, None, inplace=True)
#stellarObjects[stellarObjects.isna().any(axis=1)]

kepid        None
koi_steff    None
koi_slogg    None
koi_smet     None
koi_srad     None
koi_smass    None
koi_srho     None
koi_count    None
dtype: object

In [14]:
# Type assertion
print(stellarObjects.dtypes)

kepid          int64
koi_steff    float64
koi_slogg    float64
koi_smet     float64
koi_srad     float64
koi_smass    float64
koi_srho     float64
koi_count      int64
dtype: object


#### Koi Disposition table

The disposation table will follow the same cleaning process as the transitProperties table

In [15]:
# Make sure there are no duplicate rows based on kepid
koiObjects.drop_duplicates(subset="kepoi_name", inplace=True)

In [16]:
# Drop rows with all null values
koiObjects.dropna(how='all', inplace=True)
koiObjects.replace(np.NaN, None, inplace=True)
#koiObjects[koiObjects.isna().any(axis=1)]

kepoi_name         None
kepid              None
koi_disposition    None
koi_score          None
dtype: object

In [17]:
# Type assertion
print(koiObjects.dtypes)

kepoi_name          object
kepid                int64
koi_disposition     object
koi_score          float64
dtype: object


<hr>

### Connect to kepler object of interest (koi) database

This section require the mysql package to connect to the my sql server.

In [18]:
# Connect to MySQL Server
try:
    db = connector.connect(
        host="localhost",
        user="root",
        password="root",
        database='koi'
    )
    if db.is_connected():
        logging.info("Connected to database: {}".format(db.database))
        db_server_info = db.get_server_info()
        print("Connected to MySQL sever version", db_server_info)
        
        # Create a cursor object
        c = db.cursor()

except Error as e:
    logging.warning("Connection to database failed")
    print("Error while connecting to mySQL", e)

Connected to MySQL sever version 8.0.22


In [19]:
def execute(command, commit=False, verbose=True):
    """ Streamline the execute and fetch results with cursor object """
    c.execute(command)
    logging.debug("EXECUTE ORDER 66: ", command)
    try:
        result = c.fetchall()
        if verbose:
            pprint(result)
        return result
    except:
        if verbose:
            print("No fetched results to Print but the command worked!!")
        else:
            pass
    
    if commit:
        db.commit()
        logging.debug("COMMITED LAST COMMAND ^")
    


### Importing to MySQL Schema

The database schema was created first. Then add the tables that the pandas dataframes will be imported into.

In [20]:
stellar_table_creation = """
CREATE TABLE IF NOT EXISTS Stellar_Objects(
	kepid INT PRIMARY KEY NOT NULL,
	koi_steff FLOAT,
	koi_slogg FLOAT,
    koi_smet FLOAT,
    koi_srad FLOAT,
    koi_smass FLOAT,
    koi_srho FLOAT,
    koi_count INT
);"""
transit_table_creation = """
CREATE TABLE IF NOT EXISTS Transit_Properties(
	kepoi_name CHAR(9) PRIMARY KEY NOT NULL,
	kepid INT,
		FOREIGN KEY (kepid) REFERENCES Stellar_Objects(kepid),
	koi_period FLOAT,
    koi_prad FLOAT,
    koi_sma FLOAT,
    koi_teq FLOAT,
	koi_ror FLOAT,
    koi_dor FLOAT
);
"""
koi_table_creation = """
CREATE TABLE IF NOT EXISTS koi_disposition(
	kepoi_name CHAR(9) PRIMARY KEY NOT NULL,
	kepid INT,
		FOREIGN KEY (kepid) REFERENCES Stellar_Objects(kepid),
    koi_disposition SET('CONFIRMED', 'CANDIDATE', 'FALSE POSITIVE'),
    koi_score FLOAT
);
"""

In [21]:
logging.info("Attempting to create stellar_objects table...")
execute(stellar_table_creation, commit=True)
logging.info("\t Table created sucessfully or table already existed")

No fetched results to Print but the command worked!!


In [22]:
logging.info("Attempting to create koi_disposition table...")
execute(koi_table_creation, commit=True)
logging.info("\t Table created sucessfully or table already existed")

No fetched results to Print but the command worked!!


In [23]:
logging.info("Attempting to create transit_objects table...")
execute(transit_table_creation, commit=True)
logging.info("\t Table created sucessfully or table already existed")

No fetched results to Print but the command worked!!


In [24]:
#execute("DESCRIBE stellar_objects")

In [25]:
#execute("DESCRIBE koi_disposition")

In [26]:
#execute("DESCRIBE transit_properties")

Now we can define a function for us to import that data from the pandas dataframe into the MySQL databse.

In [27]:
def insert(dataframe, table_name, table_columns, commit_every_time=True):
    """ Takes a pandas dataframe and inserts into a MySQL table
        called table_name 
    """
    logging.info("Starting import of data to table {}...".format(table_name))
    for row in dataframe.values:
        row_tuple = str(tuple(row))
        try:
            execute("INSERT INTO {} {} VALUES {};".format(table_name, table_columns, row_tuple),\
                    commit=commit_every_time,\ # condition for commiting insertion in each 
                    verbose=False)
            logging.debug("Inserted {} into {}".format(row_tuple, table_name))
        except Error as e:
            logging.warning("Row not inserted")
    logging.info("\t Import done")

SyntaxError: unexpected character after line continuation character (<ipython-input-27-c6847adf1707>, line 10)

In [ ]:
s_cols = "(kepid, koi_steff, koi_slogg, koi_smet, koi_srad, koi_smass, koi_srho, koi_count)"
insert(stellarObjects, "stellar_objects", s_cols)

In [ ]:
t_cols = "(kepoi_name, kepid, koi_period, koi_prad, koi_sma, koi_teq, koi_ror, koi_dor)"
insert(transitProperties, "transit_properties", t_cols)

In [ ]:
d_cols = "(kepoi_name, kepid, koi_disposition, koi_score)"
insert(koiObjects, "koi_disposition", d_cols)

In [ ]:
db.commit()

In [ ]:
#execute("DELETE FROM koi_disposition")
#execute("DELETE FROM transit_properties")
#execute("DELETE FROM stellar_objects")

In [ ]:
db.close()

### References

NASA. Nasa exoplanet archive Cumulative kepler object of interest table, 2017. data retrieved from https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=koi.